# <font color='red'>듀란듀란리턴즈팀</font>의 <font color='blue'>아파트 가격예측</font>!!!

## <font color='black'>Description</font>
- 

## <font color='black'>Evaluation</font>
- <font color='red'>자전거 대여량</font>을 예측하는 <font color='blue'>회귀</font>와 관련된 문제!

In [1]:
## import pandas as pd
from datetime import datetime
import pandas as pd
import time
import numpy as np
import re
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from scipy import stats
from matplotlib import font_manager, rc

# font_name = font_manager.FontProperties(fname="/Library/Fonts/AppleGothic.ttf").get_name()
font_name = font_manager.FontProperties(fname="C:/Windows/Fonts/MALGUN.TTF").get_name()
rc('font', family=font_name)
# 노트북 안에 그래프를 그리기 위해
%matplotlib inline

# 그래프에서 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일을 사용
plt.style.use('ggplot')

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

C:\Python\anaconda3-52\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df_train = pd.read_csv('./train.csv', encoding = 'utf-8')

In [3]:
df = df_train.copy()
#convert interval unit to date
df['transaction_date'] = df.transaction_date.apply(lambda x: x[-2:])
#convert int to date
df['transaction_year_month'] = df['transaction_year_month'].astype(str)
#join month and date 
df['transaction_year_month_date'] = df[['transaction_year_month','transaction_date']].apply(lambda x: ''.join(x), axis=1)
#convert  month and date to datetime 
df['transaction_year_month_date'] = pd.to_datetime(df['transaction_year_month_date'])
#reindext datetime
# df = df.set_index(['transaction_year_month_date'])
#drop month and date


In [14]:
# df.columns

Index(['key', 'apartment_id', 'city', 'year_of_completion',
       'exclusive_use_area', 'floor', 'latitude', 'longitude',
       'address_by_law', 'total_parking_capacity_in_site',
       'total_household_count_in_sites', 'apartment_building_count_in_sites',
       'tallest_building_in_sites', 'lowest_building_in_sites', 'heat_type',
       'heat_fuel', 'room_id', 'supply_area',
       'total_household_count_of_area_type', 'room_count', 'bathroom_count',
       'front_door_structure', 'transaction_real_price',
       'transaction_year_month_date'],
      dtype='object')

In [15]:
# df = df.drop(['transaction_year_month', 'transaction_date'],axis=1)
df = pd.DataFrame(df,columns = ['key', 'apartment_id', 'city', 'year_of_completion','transaction_year_month_date',
       'exclusive_use_area', 'floor', 'latitude', 'longitude',
       'address_by_law', 'total_parking_capacity_in_site',
       'total_household_count_in_sites', 'apartment_building_count_in_sites',
       'tallest_building_in_sites', 'lowest_building_in_sites', 'heat_type',
       'heat_fuel', 'room_id', 'supply_area',
       'total_household_count_of_area_type', 'room_count', 'bathroom_count',
       'front_door_structure', 'transaction_real_price'
       ])
       

In [17]:
df.head()

,key,apartment_id,city,year_of_completion,transaction_year_month_date,exclusive_use_area,floor,latitude,longitude,address_by_law,...,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price
0,0,5584,1,1999,2006-01-20,47.43,6,37.585965,127.000231,1111017100,...,4.0,individual,gas,91120,65.63,46,1.0,1.0,corridor,215000000
1,1,5584,1,1999,2006-01-20,44.37,8,37.585965,127.000231,1111017100,...,4.0,individual,gas,91119,61.39,10,2.0,1.0,corridor,200000000
2,2,5059,1,1992,2006-01-20,54.70,8,37.580511,127.014016,1111017400,...,9.0,individual,gas,8430,72.36,201,2.0,1.0,corridor,168000000
3,3,2816,1,1993,2006-01-20,64.66,11,37.580324,127.011788,1111017400,...,11.0,individual,gas,5839,87.30,284,2.0,1.0,corridor,165000000
4,4,2816,1,1993,2006-01-20,106.62,7,37.580324,127.011788,1111017400,...,11.0,individual,gas,5836,127.74,112,4.0,2.0,stairway,280000000


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601458 entries, 0 to 1601457
Data columns (total 24 columns):
key                                   1601458 non-null int64
apartment_id                          1601458 non-null int64
city                                  1601458 non-null int64
year_of_completion                    1601458 non-null int64
exclusive_use_area                    1601458 non-null float64
floor                                 1601458 non-null int64
latitude                              1601458 non-null float64
longitude                             1601458 non-null float64
address_by_law                        1601458 non-null int64
total_parking_capacity_in_site        1509645 non-null float64
total_household_count_in_sites        1601458 non-null int64
apartment_building_count_in_sites     1601458 non-null int64
tallest_building_in_sites             1601449 non-null float64
lowest_building_in_sites              1601449 non-null float64
heat_type            

In [23]:
# 일단 뺄게요 추후 확인
# del df['transaction_year_month_date']
df.isnull().sum(axis=0)
df_train_na_zero=df.dropna(axis=0)
df_train_na_zero
df.drop(['front_door_structure','heat_type','heat_fuel'], axis=1, inplace=True)

In [45]:
# df.drop(['room_id','address_by_law'],axis =1, inplace=True)
len(df.columns)
df.transaction_year_month_date[0]

Timestamp('2006-01-20 00:00:00')

In [47]:
from datetime import datetime
import time 
Timestamp = []
for i in range(len(df.transaction_year_month_date)):
    timestamp = time.mktime(df.transaction_year_month_date[i].timetuple())
    Timestamp.append(timestamp)

In [61]:
df_2=pd.DataFrame(Timestamp)
df_2.columns=['timestamp']
df_2


df_3 = pd.concat(
    [pd.DataFrame(df), df_2],
    axis=1) 

In [67]:
# df_3.drop('transaction_year_month_date',axis = 1, inplace = True)
df_3.columns

Index(['key', 'apartment_id', 'city', 'year_of_completion',
       'exclusive_use_area', 'floor', 'latitude', 'longitude',
       'total_parking_capacity_in_site', 'total_household_count_in_sites',
       'apartment_building_count_in_sites', 'tallest_building_in_sites',
       'lowest_building_in_sites', 'supply_area',
       'total_household_count_of_area_type', 'room_count', 'bathroom_count',
       'transaction_real_price', 'timestamp'],
      dtype='object')

In [74]:
# df_train.drop("total_parking_capacity_in_site")
df = pd.DataFrame(df_3,columns = ['key', 'apartment_id', 'city', 'year_of_completion', 'timestamp',
       'exclusive_use_area', 'floor', 'latitude', 'longitude',
       'total_parking_capacity_in_site', 'total_household_count_in_sites',
       'apartment_building_count_in_sites', 'tallest_building_in_sites',
       'lowest_building_in_sites', 'supply_area',
       'total_household_count_of_area_type', 'room_count', 'bathroom_count',
       'transaction_real_price'])
df.head()

,key,apartment_id,city,year_of_completion,timestamp,exclusive_use_area,floor,latitude,longitude,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price
0,0,5584,1,1999,1.137683e+09,47.43,6,37.585965,127.000231,163.0,136,1,8.0,4.0,65.63,46,1.0,1.0,215000000
1,1,5584,1,1999,1.137683e+09,44.37,8,37.585965,127.000231,163.0,136,1,8.0,4.0,61.39,10,2.0,1.0,200000000
2,2,5059,1,1992,1.137683e+09,54.70,8,37.580511,127.014016,902.0,585,5,14.0,9.0,72.36,201,2.0,1.0,168000000
3,3,2816,1,1993,1.137683e+09,64.66,11,37.580324,127.011788,902.0,919,7,15.0,11.0,87.30,284,2.0,1.0,165000000
4,4,2816,1,1993,1.137683e+09,106.62,7,37.580324,127.011788,902.0,919,7,15.0,11.0,127.74,112,4.0,2.0,280000000


In [75]:
df.timestamp[0]
# for i in range(len(df.timestamp)):
    

1137682800.0

In [7]:
df_train_seoul=df_train[df_train['city']==1]
df_train_busan=df_train[df_train['city']==0]
df_train_seoul

,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,...,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price
0,0,5584,1,200601,11~20,1999,47.4300,6,37.585965,127.000231,...,4.0,individual,gas,91120,65.63,46,1.0,1.0,corridor,215000000
1,1,5584,1,200601,11~20,1999,44.3700,8,37.585965,127.000231,...,4.0,individual,gas,91119,61.39,10,2.0,1.0,corridor,200000000
2,2,5059,1,200601,11~20,1992,54.7000,8,37.580511,127.014016,...,9.0,individual,gas,8430,72.36,201,2.0,1.0,corridor,168000000
3,3,2816,1,200601,11~20,1993,64.6600,11,37.580324,127.011788,...,11.0,individual,gas,5839,87.30,284,2.0,1.0,corridor,165000000
4,4,2816,1,200601,11~20,1993,106.6200,7,37.580324,127.011788,...,11.0,individual,gas,5836,127.74,112,4.0,2.0,stairway,280000000
5,5,2815,1,200601,11~20,2000,84.9200,9,37.575381,126.960804,...,10.0,individual,gas,5831,109.88,454,3.0,2.0,stairway,415000000
6,6,2815,1,200601,11~20,2000,60.0000,13,37.575381,126.960804,...,10.0,individual,gas,5833,84.33,207,3.0,1.0,corridor,267000000
7,7,9867,1,200601,11~20,2005,84.7600,10,37.559200,127.019503,...,6.0,individual,gas,11862,104.30,82,3.0,2.0,stairway,415000000
8,8,2818,1,200601,11~20,1999,84.8800,18,37.555060,127.014495,...,8.0,individual,gas,5843,107.65,576,3.0,2.0,stairway,310000000
9,9,2817,1,200601,11~20,2002,59.9400,12,37.549828,127.009284,...,11.0,individual,gas,5842,85.90,864,3.0,1.0,corridor,319000000


In [8]:
df_train.corr().style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

,key,apartment_id,city,transaction_year_month,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price
key,1.0,0.19,0.083,0.99,0.16,0.071,0.033,0.082,-0.083,-0.082,0.076,0.042,0.058,0.07,0.02,0.015,0.082,-0.037,0.054,0.11,0.3
apartment_id,0.19,1.0,-0.17,0.19,0.64,0.16,0.097,-0.17,0.17,0.17,0.14,0.027,0.088,0.22,0.13,0.26,0.2,-0.18,0.1,0.35,0.16
city,0.083,-0.17,1.0,0.077,-0.078,-0.088,-0.15,1.0,-1.0,-1.0,0.036,0.1,0.15,-0.25,-0.3,-0.29,-0.092,0.023,-0.074,-0.058,0.4
transaction_year_month,0.99,0.19,0.077,1.0,0.16,0.073,0.034,0.076,-0.078,-0.077,0.078,0.042,0.059,0.071,0.022,0.015,0.084,-0.038,0.055,0.11,0.29
year_of_completion,0.16,0.64,-0.078,0.16,1.0,0.24,0.26,-0.079,0.073,0.077,0.18,0.02,0.042,0.51,0.28,0.15,0.32,-0.12,0.23,0.52,0.13
exclusive_use_area,0.071,0.16,-0.088,0.073,0.24,1.0,0.17,-0.094,0.089,0.089,0.14,-0.02,0.019,0.29,0.22,0.048,0.98,-0.15,0.77,0.65,0.52
floor,0.033,0.097,-0.15,0.034,0.26,0.17,1.0,-0.15,0.15,0.15,0.21,0.12,0.013,0.54,0.44,0.11,0.22,0.032,0.1,0.16,0.13
latitude,0.082,-0.17,1.0,0.076,-0.079,-0.094,-0.15,1.0,-0.99,-1.0,0.032,0.1,0.15,-0.25,-0.3,-0.3,-0.097,0.023,-0.079,-0.06,0.39
longitude,-0.083,0.17,-1.0,-0.078,0.073,0.089,0.15,-0.99,1.0,1.0,-0.024,-0.092,-0.14,0.26,0.31,0.29,0.093,-0.02,0.073,0.056,-0.39
address_by_law,-0.082,0.17,-1.0,-0.077,0.077,0.089,0.15,-1.0,1.0,1.0,-0.033,-0.1,-0.15,0.25,0.3,0.29,0.093,-0.023,0.075,0.058,-0.39


In [9]:
df_train_seoul.corr().style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

C:\Python\Anaconda3-52\lib\site-packages\matplotlib\colors.py:504: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,key,apartment_id,city,transaction_year_month,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price
key,1.0,0.23,nan,0.99,0.18,0.1,0.036,-0.043,-0.012,0.021,0.062,0.023,0.048,0.092,0.03,0.026,0.11,-0.045,0.084,0.14,0.27
apartment_id,0.23,1.0,nan,0.23,0.69,0.16,0.097,-0.061,-0.04,0.036,0.17,0.064,0.14,0.27,0.075,0.14,0.21,-0.17,0.15,0.4,0.27
city,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
transaction_year_month,0.99,0.23,nan,1.0,0.18,0.1,0.037,-0.046,-0.012,0.024,0.065,0.025,0.051,0.093,0.032,0.025,0.11,-0.045,0.085,0.14,0.26
year_of_completion,0.18,0.69,nan,0.18,1.0,0.17,0.22,-0.053,-0.099,-0.076,0.16,-0.016,0.034,0.52,0.16,0.095,0.25,-0.16,0.21,0.52,0.11
exclusive_use_area,0.1,0.16,nan,0.1,0.17,1.0,0.11,-0.16,-0.024,0.074,0.074,-0.06,0.015,0.22,0.13,0.039,0.98,-0.12,0.76,0.64,0.64
floor,0.036,0.097,nan,0.037,0.22,0.11,1.0,-0.069,0.036,0.023,0.2,0.12,0.066,0.45,0.33,0.052,0.14,0.035,0.075,0.13,0.16
latitude,-0.043,-0.061,nan,-0.046,-0.053,-0.16,-0.069,1.0,0.28,-0.65,-0.078,-0.012,-0.076,-0.14,-0.14,-0.066,-0.16,0.068,-0.14,-0.079,-0.37
longitude,-0.012,-0.04,nan,-0.012,-0.099,-0.024,0.036,0.28,1.0,-0.016,0.18,0.19,0.14,0.077,0.064,-0.067,-0.024,0.11,-0.041,-0.052,0.1
address_by_law,0.021,0.036,nan,0.024,-0.076,0.074,0.023,-0.65,-0.016,1.0,0.2,0.16,0.19,0.055,0.1,0.024,0.065,0.026,0.041,0.0063,0.32


In [10]:
df_train_busan.corr().style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

C:\Python\Anaconda3-52\lib\site-packages\matplotlib\colors.py:504: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,key,apartment_id,city,transaction_year_month,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price
key,1.0,0.19,nan,0.99,0.15,0.045,0.062,0.02,-0.0047,0.044,0.1,0.064,0.044,0.11,0.072,0.078,0.061,-0.029,0.014,0.071,0.38
apartment_id,0.19,1.0,nan,0.19,0.54,0.12,0.048,0.058,-0.044,0.075,0.096,0.0024,0.044,0.13,0.1,0.39,0.16,-0.19,-0.0042,0.25,0.25
city,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
transaction_year_month,0.99,0.19,nan,1.0,0.15,0.047,0.06,0.02,-0.0052,0.046,0.1,0.061,0.043,0.11,0.071,0.076,0.064,-0.03,0.015,0.071,0.38
year_of_completion,0.15,0.54,nan,0.15,1.0,0.34,0.3,0.059,0.039,0.075,0.27,0.15,0.13,0.55,0.43,0.21,0.41,-0.057,0.25,0.51,0.47
exclusive_use_area,0.045,0.12,nan,0.047,0.34,1.0,0.21,-0.038,0.078,0.014,0.28,0.092,0.092,0.34,0.27,-0.0042,0.99,-0.19,0.79,0.67,0.71
floor,0.062,0.048,nan,0.06,0.3,0.21,1.0,0.0015,0.13,-0.084,0.27,0.18,-0.02,0.58,0.48,0.11,0.27,0.038,0.12,0.19,0.42
latitude,0.02,0.058,nan,0.02,0.059,-0.038,0.0015,1.0,0.43,0.31,-0.052,-0.046,-0.035,0.0011,0.012,-0.003,-0.031,-0.11,-0.026,-0.0024,0.013
longitude,-0.0047,-0.044,nan,-0.0052,0.039,0.078,0.13,0.43,1.0,0.11,0.039,-0.052,-0.12,0.23,0.23,-0.025,0.092,-0.13,0.059,0.029,0.22
address_by_law,0.044,0.075,nan,0.046,0.075,0.014,-0.084,0.31,0.11,1.0,-0.066,-0.058,0.048,-0.14,-0.12,-0.02,0.011,-0.018,0.032,0.024,-0.02


In [41]:
df_train_na_zero=df_train_na_zero.dropna(axis=0)
df_train_na_zero.isna().sum()

key                                   0
apartment_id                          0
city                                  0
transaction_year_month                0
transaction_date                      0
year_of_completion                    0
exclusive_use_area                    0
floor                                 0
latitude                              0
longitude                             0
address_by_law                        0
total_parking_capacity_in_site        0
total_household_count_in_sites        0
apartment_building_count_in_sites     0
tallest_building_in_sites             0
lowest_building_in_sites              0
room_id                               0
supply_area                           0
total_household_count_of_area_type    0
room_count                            0
bathroom_count                        0
transaction_real_price                0
dtype: int64

In [42]:
set(df_train_na_zero['heat_type'])

KeyError: 'heat_type'

In [ ]:
# df_train_na_zero['heat_type']=='central'

# pd.get_dummies로 One-hot 인코딩하자 듀란!
heat_type_one_hot_encoded = pd.get_dummies(df_train_na_zero.heat_type) 

# 원래 데이터와 비교식으로 보여주기용 데이터프레임
df_heat_type_with_onehot = pd.concat(
    [pd.DataFrame(df_train_na_zero.heat_type), heat_type_one_hot_encoded],
    axis=1) 
df_heat_type_with_onehot.head(10)

In [ ]:
# df_train_na_zero['heat_type']=='central'

# pd.get_dummies로 One-hot 인코딩하자 듀란!
heat_fuel_one_hot_encoded = pd.get_dummies(df_train_na_zero.heat_fuel)

# 원래 데이터와 비교식으로 보여주기용 데이터프레임
df_heat_fuel_with_onehot = pd.concat(
    [pd.DataFrame(df_train_na_zero.heat_fuel), heat_fuel_one_hot_encoded],
    axis=1) 
df_heat_fuel_with_onehot.head(10)

In [43]:
# df_train_na_zero['heat_type']=='central'

# pd.get_dummies로 One-hot 인코딩하자 듀란!
front_door_structure_one_hot_encoded = pd.get_dummies(df_train_na_zero.front_door_structure) 

# 원래 데이터와 비교식으로 보여주기용 데이터프레임
df_front_door_structure_with_onehot = pd.concat(
    [pd.DataFrame(df_train_na_zero.front_door_structure), front_door_structure_one_hot_encoded],
    axis=1) 
df_front_door_structure_with_onehot.head(10)

AttributeError: 'DataFrame' object has no attribute 'front_door_structure'

In [44]:
pd.concat([df_train_na_zero,df_heat_type_with_onehot,df_heat_fuel_with_onehot,df_front_door_structure_with_onehot], axis=1)

,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,...,individual,heat_fuel,-,cogeneration,gas,front_door_structure,-,corridor,mixed,stairway
0,0,5584,1,200601,11~20,1999,47.4300,6,37.585965,127.000231,...,1,gas,0,0,1,corridor,0,1,0,0
1,1,5584,1,200601,11~20,1999,44.3700,8,37.585965,127.000231,...,1,gas,0,0,1,corridor,0,1,0,0
2,2,5059,1,200601,11~20,1992,54.7000,8,37.580511,127.014016,...,1,gas,0,0,1,corridor,0,1,0,0
3,3,2816,1,200601,11~20,1993,64.6600,11,37.580324,127.011788,...,1,gas,0,0,1,corridor,0,1,0,0
4,4,2816,1,200601,11~20,1993,106.6200,7,37.580324,127.011788,...,1,gas,0,0,1,stairway,0,0,0,1
5,5,2815,1,200601,11~20,2000,84.9200,9,37.575381,126.960804,...,1,gas,0,0,1,stairway,0,0,0,1
6,6,2815,1,200601,11~20,2000,60.0000,13,37.575381,126.960804,...,1,gas,0,0,1,corridor,0,1,0,0
7,7,9867,1,200601,11~20,2005,84.7600,10,37.559200,127.019503,...,1,gas,0,0,1,stairway,0,0,0,1
8,8,2818,1,200601,11~20,1999,84.8800,18,37.555060,127.014495,...,1,gas,0,0,1,stairway,0,0,0,1
9,9,2817,1,200601,11~20,2002,59.9400,12,37.549828,127.009284,...,1,gas,0,0,1,corridor,0,1,0,0


In [49]:
df_train_na_zero.head()

,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,...,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price
0,0,5584,1,200601,11~20,1999,47.43,6,37.585965,127.000231,...,136,1,8.0,4.0,91120,65.63,46,1.0,1.0,215000000
1,1,5584,1,200601,11~20,1999,44.37,8,37.585965,127.000231,...,136,1,8.0,4.0,91119,61.39,10,2.0,1.0,200000000
2,2,5059,1,200601,11~20,1992,54.70,8,37.580511,127.014016,...,585,5,14.0,9.0,8430,72.36,201,2.0,1.0,168000000
3,3,2816,1,200601,11~20,1993,64.66,11,37.580324,127.011788,...,919,7,15.0,11.0,5839,87.30,284,2.0,1.0,165000000
4,4,2816,1,200601,11~20,1993,106.62,7,37.580324,127.011788,...,919,7,15.0,11.0,5836,127.74,112,4.0,2.0,280000000


In [51]:
# del df_train_na_zero['hea']
# del df_train_na_zero['heat_type']
# del df_train_na_zero['front_door_structure']
del df_train_na_zero['transaction_date']


In [19]:
# type(df_train_na_zero['transaction_year_month_date'][0])

KeyError: 'transaction_year_month_date'

In [ ]:
# import time

In [ ]:
# df_train_na_zero['transaction_year_month_date']=time.time()

In [ ]:
# df_train_na_zero['transaction_year_month_date']=str(df_train_na_zero['transaction_year_month_date'])

In [52]:
df_train_na_zero.to_csv('./data/듀1은잠을들거라.csv', sep=',', encoding='utf-8')

In [47]:
from tqdm import tqdm_notebook
import tensorflow as tf
import random
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
len(df_train_na_zero.columns)

In [58]:
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # for reproducibility

# np.loadtxt('./data/test2_bike.csv', delimiter=',', dtype=np.float32)
xy = np.loadtxt('./data/듀1은잠을들거라.csv', delimiter=',', skiprows=1, dtype=np.float64)
x_data = xy[:, 1:-1]
y_data = xy[:, [-1]]

# Make sure the shape and data are OK
print("x_data.shape : {}, \tlen(x_data) : {} \nx_data : \n{}".format(x_data.shape, len(x_data), x_data))
print("-"*25)
print("y_data.shape : {}  \ny_data : \n{}".format(y_data.shape, y_data))

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 20])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W1 = tf.Variable(tf.random_normal([20, 128]), name='weight')
b1 = tf.Variable(tf.random_normal([128]), name='bias')
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

# W2 = tf.Variable(tf.random_normal([128, 128]), name='weight2')
# b2 = tf.Variable(tf.random_normal([128]), name='bias2')
# L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W2 = tf.Variable(tf.random_normal([128, 1]))
b2 = tf.Variable(tf.random_normal([1]))

# Hypothesis
hypothesis = tf.matmul(L1, W2) + b2

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

x_data.shape : (1485957, 20), 	len(x_data) : 1485957 
x_data : 
[[0.000000e+00 5.584000e+03 1.000000e+00 ... 4.600000e+01 1.000000e+00
  1.000000e+00]
 [1.000000e+00 5.584000e+03 1.000000e+00 ... 1.000000e+01 2.000000e+00
  1.000000e+00]
 [2.000000e+00 5.059000e+03 1.000000e+00 ... 2.010000e+02 2.000000e+00
  1.000000e+00]
 ...
 [1.605356e+06 2.224300e+04 0.000000e+00 ... 2.090000e+02 3.000000e+00
  2.000000e+00]
 [1.605366e+06 3.686000e+03 1.000000e+00 ... 4.000000e+00 3.000000e+00
  1.000000e+00]
 [1.605373e+06 2.937000e+03 1.000000e+00 ... 2.090000e+02 3.000000e+00
  2.000000e+00]]
-------------------------
y_data.shape : (1485957, 1)  
y_data : 
[[2.15e+08]
 [2.00e+08]
 [1.68e+08]
 ...
 [3.43e+08]
 [4.85e+08]
 [4.30e+08]]


In [ ]:

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-3)
train = optimizer.minimize(cost)

In [62]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in tqdm_notebook(range(10001)):
    cost_val, hy_val, t_val = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})

    if step % 100 == 0 or step < 100 : 
        print("\nStep : {} \nCost : {} \nPrediction :\n{}".format(step, cost_val, t_val))
        print("Step : {} \tCost : {} ".format(step, cost_val))


Step : 0 
Cost : 1.953849974646322e+19 
Prediction :
None
Step : 0 	Cost : 1.953849974646322e+19 

Step : 1 
Cost : inf 
Prediction :
None
Step : 1 	Cost : inf 


KeyboardInterrupt: 

# 상관계수&시각화를 통해 feature 조정

In [ ]:
df_train["year"] = df_train["datetime"].dt.year
df_train["month"] = df_train["datetime"].dt.month
df_train["day"] = df_train["datetime"].dt.day
df_train["hour"] = df_train["datetime"].dt.hour
df_train.shape

In [ ]:
df_train.corr().style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

In [ ]:
del df_train["casual"]
del df_train["registered"]
del df_train["season"]
del df_train['atemp']

In [ ]:
mask = np.array(df_train.corr()) 
mask[np.tril_indices_from(mask)] = False 
fig, ax = plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(df_train.corr(), mask=mask,vmax=.8, square=True,annot=True)

In [ ]:
import seaborn as sns
figure, ((ax1,ax2), (ax3,ax4)) = plt.subplots(nrows=2, ncols=2)
figure.set_size_inches(18,8)

sns.barplot(data=df_train, x="year", y="count", ax=ax1)
sns.barplot(data=df_train, x="month", y="count", ax=ax2)
sns.barplot(data=df_train, x="day", y="count", ax=ax3)
sns.barplot(data=df_train, x="hour", y="count", ax=ax4)

ax1.set(ylabel='Count',title="연도별 대여량")
ax2.set(xlabel='month',title="월별 대여량")
ax3.set(xlabel='day', title="일별 대여량")
ax4.set(xlabel='hour', title="시간별 대여량")


- 연도별 자전거 대여량 2011 < <font color='red'>2012</font>
- 월별 자전거 대여량은 < <font color='red'>6월</font>에 가장 많고 7~10월도 대여량이 많다. 그리고 <font color='blue'>1월</font>에 가장 적다.
- 일별자전거 대여량은 1일부터 19일까지만 있고 나머지 날짜는 test.csv에 있다. 
  그래서 이 데이터를 가지고 바로 학습시킬 수는 없음.
- 시간 대 대여량을 보면 <font color='red'>출퇴근 시간</font>에 대여량이 많은 것 같다.

In [ ]:
max(df_train['count']),min(df_train['count'])

In [ ]:
# trainWithoutOutliers
df_trainWithoutOutliers = df_train[np.abs(df_train["count"] - df_train["count"].mean()) <= (3*df_train["count"].std())]

print(df_train.shape) #outlier 제거 전
print(df_trainWithoutOutliers.shape) #outlier제거 후 

## 풍속 결측치 조정

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3)
fig.set_size_inches(12, 5)
sns.regplot(x="temp", y="count", data=df_train,ax=ax1)
sns.regplot(x="windspeed", y="count", data=df_train,ax=ax2)
sns.regplot(x="humidity", y="count", data=df_train,ax=ax3)

- 풍속의 경우 0에 숫자가 몰려 있는 것을 볼 수 있음
- 결측치라는 것을 알 수 있음

In [ ]:
# windspeed가 0인 결측값이 존재!!! 
df_trainWithoutOutliers['windspeed'].value_counts().head()

In [ ]:
# 풍속이 0인것과 아닌 것의 세트 나누기
trainWind0 = df_trainWithoutOutliers.loc[df_trainWithoutOutliers['windspeed'] == 0]
trainWindNot0 = df_trainWithoutOutliers.loc[df_trainWithoutOutliers['windspeed'] != 0]

In [ ]:
del trainWind0['windspeed']

In [ ]:
# 결측값 갯수 총 1297개
len(trainWind0)

In [ ]:
# 결측값을 채워넣기 위해 trainWindNot0을 이용해 랜덤값을 뽑자!!!
wind=np.random.normal(trainWindNot0['windspeed'].mean(), 1, 1297)
df_wind=pd.DataFrame(wind)
df_wind=df_wind.rename(columns={0:'windspeed'})
df_wind.head()

In [ ]:
trainWind0 = trainWind0.reset_index(drop=True)
trainWind0.columns

In [ ]:
trainWind0 = pd.concat([trainWind0,df_wind],axis = 1)
trainWind0.head()

In [ ]:
train_all = pd.concat([trainWind0, trainWindNot0], axis=0)

In [ ]:
train_all=train_all[[ 'holiday', 'workingday', 'weather', 'temp', 
        'humidity', 'windspeed', 'year', 'month',
       'day', 'hour', 'count']]
train_all.head()

# 다른 feature 값들 정규화

In [ ]:
# train_all
train_all['year'] =train_all['year'] -2011
train_all['year']

train_all.head()

In [ ]:
train_all.describe()

In [ ]:
train_all.to_csv('./data/듀란이들의bike_copy1.csv', sep=',', encoding='utf-8')

# <font color='red'>Learning</font> by <font color='blue'>doing</font>, <font color='blue'>Doing</font> by <font color='red'>learning!</font>

In [ ]:
from tqdm import tqdm_notebook
import tensorflow as tf
import random
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # for reproducibility

# np.loadtxt('./data/test2_bike.csv', delimiter=',', dtype=np.float32)
xy = np.loadtxt('./data/듀란이들의bike_copy1.csv', delimiter=',', skiprows=1, dtype=np.int32)
x_data = xy[:, 1:-1]
y_data = xy[:, [-1]]

# Make sure the shape and data are OK
print("x_data.shape : {}, \tlen(x_data) : {} \nx_data : \n{}".format(x_data.shape, len(x_data), x_data))
print("-"*25)
print("y_data.shape : {}  \ny_data : \n{}".format(y_data.shape, y_data))

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 10])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W1 = tf.Variable(tf.random_normal([10, 128]), name='weight')
b1 = tf.Variable(tf.random_normal([128]), name='bias')
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

# W2 = tf.Variable(tf.random_normal([128, 128]), name='weight2')
# b2 = tf.Variable(tf.random_normal([128]), name='bias2')
# L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W2 = tf.Variable(tf.random_normal([128, 1]))
b2 = tf.Variable(tf.random_normal([1]))

# Hypothesis
hypothesis = tf.matmul(L1, W2) + b2

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-7)
train = optimizer.minimize(cost)


In [ ]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in tqdm_notebook(range(10001)):
    cost_val, hy_val, t_val = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})

    if step % 100 == 0 or step < 100 : 
        print("\nStep : {} \nCost : {} \nPrediction :\n{}".format(step, cost_val, t_val))
        print("Step : {} \tCost : {} ".format(step, cost_val))
        